In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=e79387ade992f8fac4bae8d1b180c36fba9e0485aa01fc5b1df970984d64877e
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Read Top 5 Rows from CSV").getOrCreate()
df = spark.read.csv("housing.csv", header = True)
df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [ ]:
#creating table using createOrReplaceTempview
df.createOrReplaceTempView("myTable")

In [ ]:
top5Rows = spark.sql("SELECT * FROM myTable LIMIT 5")
top5Rows.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [ ]:
#1. What is the minimum income
minimumIncome = spark.sql("select MIN(median_income)as Minimun_Income from myTable")
minimumIncome.show()

+--------------+
|Minimun_Income|
+--------------+
|        0.4999|
+--------------+



In [ ]:
#2.What is the maximum income
spark.sql('select * from myTable where median_income = (select max(median_income) from myTable)').show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -117.93|    33.9|              30.0|     2629.0|         331.0|     956.0|     319.0|       9.9071|          500001.0|      <1H OCEAN|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [ ]:
#3.What is the average income
avgIncome = spark.sql("select avg(median_income)as avg_Income from myTable")
avgIncome.show()

+------------------+
|        avg_Income|
+------------------+
|3.8706710029070246|
+------------------+



In [ ]:
#4. What is Average income between from Lowest to average income
avg_income_between=spark.sql("SELECT AVG(median_income) as Average_Income_between FROM myTable WHERE median_income <=(SELECT AVG(median_income)FROM myTable) AND median_income >=(SELECT MIN(median_income) FROM myTable)")
avg_income_between.show()

+----------------------+
|Average_Income_between|
+----------------------+
|    2.6656849300233136|
+----------------------+



In [ ]:
#5.What is average icome from average income to maximum one
avg_income_between_higher=spark.sql("SELECT AVG(median_income) as Average_Income_between_avgtomax FROM myTable WHERE median_income >=(SELECT AVG(median_income)FROM myTable) AND median_income <=(SELECT MAX(median_income) FROM myTable)")
avg_income_between_higher.show()

+-------------------------------+
|Average_Income_between_avgtomax|
+-------------------------------+
|              5.545596062992115|
+-------------------------------+



In [ ]:
#Task- Check how many duplicate latitude presents and show me the list of rows which does not have
#duplicate latitude

duplicate_latitudes_count = spark.sql("SELECT latitude, COUNT(*) as count FROM myTable GROUP BY latitude HAVING count > 1").count()
duplicate_latitudes_count

779

In [ ]:
non_duplicate_df = spark.sql("SELECT * FROM myTable WHERE latitude NOT IN (SELECT latitude FROM myTable GROUP BY latitude HAVING COUNT(*) > 1)")

In [ ]:
non_duplicate_df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|   -121.8|   39.64|              25.0|     2202.0|         422.0|    1109.0|     403.0|       2.8306|           87500.0|         INLAND|
|  -121.56|   39.69|               8.0|     2836.0|         522.0|    1163.0|     512.0|         3.13|          168300.0|         INLAND|
|  -121.46|   39.54|              14.0|     5549.0|        1000.0|    1822.0|     919.0|       2.9562|          142300.0|         INLAND|
|  -121.74|   39.38|              27.0|     2596.0|         435.0|    1100.0|     409.0|       2.3243|           85500.0|         INLAND|
|  -124.21|   41.77|              

In [ ]:
#Task 2- on Which latitude and longitude has minimum population
min_population_row = spark.sql("SELECT latitude, longitude, population FROM myTable ORDER BY population ASC LIMIT 1")

min_population_row.show()

+--------+---------+----------+
|latitude|longitude|population|
+--------+---------+----------+
|   37.95|  -122.37|     100.0|
+--------+---------+----------+



In [ ]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]
columns = ["firstname","middlename","lastname","dob","gender","salary"]
simple_df = spark.createDataFrame(data,  schema=columns)
simple_df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [ ]:
#creating table using createOrReplaceTempview
simple_df.createOrReplaceTempView("dataTable")

In [ ]:
new_df = spark.sql("SELECT *, CASE WHEN salary < 4000 THEN True Else False End as increment FROM dataTable")
new_df.show()

+---------+----------+--------+----------+------+------+---------+
|firstname|middlename|lastname|       dob|gender|salary|increment|
+---------+----------+--------+----------+------+------+---------+
|    James|          |   Smith|1991-04-01|     M|  3000|     true|
|  Michael|      Rose|        |2000-05-19|     M|  4000|    false|
|   Robert|          |Williams|1978-09-05|     M|  4000|    false|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|    false|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|     true|
+---------+----------+--------+----------+------+------+---------+

